<a href="https://colab.research.google.com/github/GauravN03/github_practice_arcap_new/blob/master/Email_spam_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('/content/spam_assassin.csv')
df.head()

,text,target
0,From ilug-admin@linux.ie Mon Jul 29 11:28:02 2...,0
1,From gort44@excite.com Mon Jun 24 17:54:21 200...,1
2,From fork-admin@xent.com Mon Jul 29 11:39:57 2...,1
3,From dcm123@btamail.net.cn Mon Jun 24 17:49:23...,1
4,From ilug-admin@linux.ie Mon Aug 19 11:02:47 2...,0


In [4]:
import re

In [6]:
keywords = ['free', 'win', 'winner', 'cash', 'money', 'urgent', 'limited time', 'click here', 'subscribe', 'unsubscribe']

for keyword in keywords:
  df[keyword] = df['text'].apply(lambda x: 1 if pd.notna(x) and re.search(r'\b' + re.escape(keyword) + r'\b', x, re.IGNORECASE) else 0)

df['num_links'] = df['text'].apply(lambda x: len(re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', x)) if pd.notna(x) else 0)
df['has_exclamation'] = df['text'].apply(lambda x: 1 if pd.notna(x) and re.search(r'!', x) else 0)

display(df.head())

,text,target,free,win,winner,cash,money,urgent,limited time,click here,subscribe,unsubscribe,num_links,has_exclamation
0,From ilug-admin@linux.ie Mon Jul 29 11:28:02 2...,0,0,0,0,0,0,0,0,0,0,0,2,1
1,From gort44@excite.com Mon Jun 24 17:54:21 200...,1,1,0,0,1,0,0,0,0,0,0,2,1
2,From fork-admin@xent.com Mon Jul 29 11:39:57 2...,1,1,0,0,0,0,0,0,0,1,1,7,1
3,From dcm123@btamail.net.cn Mon Jun 24 17:49:23...,1,1,0,0,0,0,0,0,0,0,0,2,0
4,From ilug-admin@linux.ie Mon Aug 19 11:02:47 2...,0,0,0,0,0,0,0,0,0,0,0,1,0


**Reasoning**:
Check for missing values and split the data into training and testing sets.



In [7]:
df.info(())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5796 entries, 0 to 5795
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             5796 non-null   object
 1   target           5796 non-null   int64 
 2   free             5796 non-null   int64 
 3   win              5796 non-null   int64 
 4   winner           5796 non-null   int64 
 5   cash             5796 non-null   int64 
 6   money            5796 non-null   int64 
 7   urgent           5796 non-null   int64 
 8   limited time     5796 non-null   int64 
 9   click here       5796 non-null   int64 
 10  subscribe        5796 non-null   int64 
 11  unsubscribe      5796 non-null   int64 
 12  num_links        5796 non-null   int64 
 13  has_exclamation  5796 non-null   int64 
dtypes: int64(13), object(1)
memory usage: 634.1+ KB


In [8]:
df.describe()

,target,free,win,winner,cash,money,urgent,limited time,click here,subscribe,unsubscribe,num_links,has_exclamation
count,5796.000000,5796.000000,5796.000000,5796.000000,5796.000000,5796.000000,5796.000000,5796.00000,5796.000000,5796.000000,5796.000000,5796.000000,5796.000000
mean,0.327122,0.226536,0.019324,0.003796,0.041925,0.103002,0.015355,0.01363,0.128537,0.425466,0.491546,4.454106,0.499137
std,0.469203,0.418626,0.137672,0.061498,0.200436,0.303988,0.122973,0.11596,0.334716,0.494456,0.499972,11.058998,0.500042
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,3.000000,0.000000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.000000,1.000000,5.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,517.000000,1.000000


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [10]:
X=df.drop(['text','target'],axis=1)
y=df['target']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
log_regg=LogisticRegression()
log_regg.fit(X_train,y_train)

LogisticRegression()

In [15]:
logg_regg_predict=log_regg.predict(X_test)
logistic_testing_score=accuracy_score(logg_regg_predict,y_test)
print(f"the testing accuracy of logistic reggresion model is : {logistic_testing_score}")
logistic_trainning_score=log_regg.score(X_train,y_train)
print(f"the trainning accuracy of logistic reggresion model is : {logistic_trainning_score}")

the testing accuracy of logistic reggresion model is : 0.8827586206896552
the trainning accuracy of logistic reggresion model is : 0.8796376186367558


In [16]:
from xgboost import XGBClassifier
xgb_model=xgb_model=XGBClassifier()
xgb_model.fit(X_train,y_train)

xgb_trainning_score=accuracy_score(xgb_model.predict(X_train),y_train)
print(f"the training accuracy of xgboost model is : {xgb_trainning_score}")

xgb_testing_score=accuracy_score(xgb_model.predict(X_test),y_test)
print(f"the testing accuracy of xgboost model is :{xgb_testing_score}")

the training accuracy of xgboost model is : 0.9139344262295082
the testing accuracy of xgboost model is :0.903448275862069


In [19]:
from sklearn.ensemble import RandomForestClassifier
rf_model=RandomForestClassifier()
rf_model.fit(X_train,y_train)

rf_trainning_score=accuracy_score(rf_model.predict(X_train),y_train)
print(f"the training accuracy of random forest model is : {rf_trainning_score}")

rf_testing_score=accuracy_score(rf_model.predict(X_test),y_test)
print(f"the testing accuracy of random forest model is :{rf_testing_score}")

the training accuracy of random forest model is : 0.9180327868852459
the testing accuracy of random forest model is :0.9025862068965518


In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model=Sequential([tf.Dense()])

In [26]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation='sigmoid') # Output layer for binary classification
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Now I'll compile the model.

In [27]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,905 (15.25 KB)

 Trainable params: 3,681 (14.38 KB)

 Non-trainable params: 224 (896.00 B)

In [29]:
model.fit(X_train, y_train, epochs=30, batch_size=len(X_train))

Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.8947 - loss: 0.2616
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.8980 - loss: 0.2610
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.8993 - loss: 0.2609
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.8956 - loss: 0.2623
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.8926 - loss: 0.2666
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.8950 - loss: 0.2653
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.8960 - loss: 0.2653
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.8978 - loss: 0.2623
Epoch 9/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.8984 - loss: 0.2598
Epoch 10/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.8956 - loss: 0.2611
Epoch 11/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.8999 - loss: 0.2612
Epoch 12/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.8950 - loss: 0.2